In [ ]:
#!pip install pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.2 MB/s eta 0:00:00


In [65]:
import numpy as np
import pandas as pd
import os,sys,glob,json
import matplotlib.pyplot as plt
import nibabel as nib
import pydicom

%matplotlib inline


In [7]:
STUDY_PATH=os.path.join('/NAS','dumbo','protocoles','THY1')
SCRIPT_PATH=os.path.join(STUDY_PATH,'scripts')
DATA_PATH=os.path.join(STUDY_PATH,'u1172_thy1_devos_cedrick')

# copies des images dans les repertoires réel et imaginaires

In [42]:
directories = [d for d in os.listdir(DATA_PATH) if os.path.isdir(os.path.join(DATA_PATH, d))]
for directory in directories: 
    result_list = []
    result_list.extend([d for d in os.listdir(os.path.join(DATA_PATH, directory, 'T0', 'IRM', 'dicom')) if '3DMGET2' in d])
    result_list.sort()
    print(f'{directory} magn : {result_list[0]}')
    print(f'{directory} reel : {result_list[1]}')
    print(f'{directory} ima : {result_list[2]}')

    os.makedirs(os.path.join(STUDY_PATH, 'processed_data', directory, 'QSM'), exist_ok=True)
    src = os.path.join(DATA_PATH, directory, 'T0', 'IRM', 'dicom', result_list[1])
    dst = os.path.join(STUDY_PATH, 'processed_data', directory, 'QSM', 'reel')
    os.system(f'cp -r {src} {dst}')

    src = os.path.join(DATA_PATH, directory, 'T0', 'IRM', 'dicom', result_list[2])
    dst = os.path.join(STUDY_PATH, 'processed_data', directory, 'QSM', 'imaginaire')
    os.system(f'cp -r {src} {dst}')

20250212_BW1293 magn : 3DMGET2stardef13_100u
20250212_BW1293 reel : 3DMGET2stardef13_100u_reconstruction_P6
20250212_BW1293 ima : 3DMGET2stardef13_100u_reconstruction_P7
20241018_BW1294 magn : 3DMGET2stardef13_100u
20241018_BW1294 reel : 3DMGET2stardef13_100u_reconstruction_P4
20241018_BW1294 ima : 3DMGET2stardef13_100u_reconstruction_P5
20241223_BW1234 magn : 3DMGET2stardef13_100u
20241223_BW1234 reel : 3DMGET2stardef13_100u_reconstruction_P4
20241223_BW1234 ima : 3DMGET2stardef13_100u_reconstruction_P5
20250129_BW1271 magn : 3DMGET2stardef13_100u
20250129_BW1271 reel : 3DMGET2stardef13_100u_reconstruction_P10
20250129_BW1271 ima : 3DMGET2stardef13_100u_reconstruction_P11
20241126_BW1248 magn : 3DMGET2stardef13_100u
20241126_BW1248 reel : 3DMGET2stardef13_100u_reconstruction_P8
20241126_BW1248 ima : 3DMGET2stardef13_100u_reconstruction_P9
20250212_BW1301 magn : 3DMGET2stardef13_100u
20250212_BW1301 reel : 3DMGET2stardef13_100u_reconstruction_P6
20250212_BW1301 ima : 3DMGET2stardef13_1

# creation des images de phase à partir des images nifti reel et imaginaires 

In [69]:
directories = [d for d in os.listdir(DATA_PATH) if os.path.isdir(os.path.join(DATA_PATH, d))]
for directory in directories: # ['20250212_BW1293']: # 20241220_BW1311
    for echo in ['e1','e1a','e1b','e1c','e1d','e1e']:
        nifti_output_path = os.path.join(STUDY_PATH, 'processed_data',directory,'QSM',f'phase_{echo}.nii.gz')
        if not os.path.exists(nifti_output_path):
            files = glob.glob(os.path.join(DATA_PATH, directory,f'*3DMGET2star*_{echo}.nii.gz'))
            files.sort()
            reel=nib.load(files[1])
            imaginaire=nib.load(files[2])
            phase_images = np.arctan2(reel.get_fdata(),imaginaire.get_fdata())
            # Enregistrer l'image NIfTI
            nifti_image = nib.Nifti1Image(phase_images,reel.affine, reel.header)
            nib.save(nifti_image, nifti_output_path)
            print(f'Les images de phase ont été enregistrées dans {nifti_output_path}')

Les images de phase ont été enregistrées dans /NAS/dumbo/protocoles/THY1/processed_data/20250212_BW1293/QSM/phase_e1.nii.gz
Les images de phase ont été enregistrées dans /NAS/dumbo/protocoles/THY1/processed_data/20250212_BW1293/QSM/phase_e1a.nii.gz
Les images de phase ont été enregistrées dans /NAS/dumbo/protocoles/THY1/processed_data/20250212_BW1293/QSM/phase_e1b.nii.gz
Les images de phase ont été enregistrées dans /NAS/dumbo/protocoles/THY1/processed_data/20250212_BW1293/QSM/phase_e1c.nii.gz
Les images de phase ont été enregistrées dans /NAS/dumbo/protocoles/THY1/processed_data/20250212_BW1293/QSM/phase_e1d.nii.gz
Les images de phase ont été enregistrées dans /NAS/dumbo/protocoles/THY1/processed_data/20250212_BW1293/QSM/phase_e1e.nii.gz


In [76]:
import nibabel as nib
import numpy as np

directories = [d for d in os.listdir(DATA_PATH) if os.path.isdir(os.path.join(DATA_PATH, d))]
for directory in directories: # ['20250212_BW1293']: # 20241220_BW1311
    for echo in ['e1','e1a','e1b','e1c','e1d','e1e']:
        nifti_label= os.path.join(STUDY_PATH, 'processed_data',directory,'label','sphinx_N43DT2_label_NUS_RL_STEPS_5_8.nii.gz')
        nifti_R2= os.path.join(STUDY_PATH, 'processed_data',directory,'R2star','R2map_s-1.nii.gz')
        print(f'mri_convert -rt nearest -rl {nifti_label} {nifti_R2} {nifti_R2.replace(".nii.gz","_gl.nii.gz")}')
        # Charger les fichiers NIfTI

mri_convert -rt nearest -rl /NAS/dumbo/protocoles/THY1/processed_data/20250212_BW1293/label/sphinx_N43DT2_label_NUS_RL_STEPS_5_8.nii.gz /NAS/dumbo/protocoles/THY1/processed_data/20250212_BW1293/R2star/R2map_s-1.nii.gz /NAS/dumbo/protocoles/THY1/processed_data/20250212_BW1293/R2star/R2map_s-1_gl.nii.gz
mri_convert -rt nearest -rl /NAS/dumbo/protocoles/THY1/processed_data/20250212_BW1293/label/sphinx_N43DT2_label_NUS_RL_STEPS_5_8.nii.gz /NAS/dumbo/protocoles/THY1/processed_data/20250212_BW1293/R2star/R2map_s-1.nii.gz /NAS/dumbo/protocoles/THY1/processed_data/20250212_BW1293/R2star/R2map_s-1_gl.nii.gz
mri_convert -rt nearest -rl /NAS/dumbo/protocoles/THY1/processed_data/20250212_BW1293/label/sphinx_N43DT2_label_NUS_RL_STEPS_5_8.nii.gz /NAS/dumbo/protocoles/THY1/processed_data/20250212_BW1293/R2star/R2map_s-1.nii.gz /NAS/dumbo/protocoles/THY1/processed_data/20250212_BW1293/R2star/R2map_s-1_gl.nii.gz
mri_convert -rt nearest -rl /NAS/dumbo/protocoles/THY1/processed_data/20250212_BW1293/label

In [81]:
import nibabel as nib
import numpy as np

results = []
directories = [d for d in os.listdir(DATA_PATH) if os.path.isdir(os.path.join(DATA_PATH, d))]
for directory in directories: # ['20250212_BW1293']: # 20241220_BW1311
    nifti_label= os.path.join(STUDY_PATH, 'processed_data',directory,'label','sphinx_N43DT2_label_NUS_RL_STEPS_5_8.nii.gz')
    nifti_R2= os.path.join(STUDY_PATH, 'processed_data',directory,'R2star','R2map_s-1_gl.nii.gz')

    # Charger les fichiers NIfTI
    label_img = nib.load(nifti_label)
    R2_img = nib.load(nifti_R2)

    # Obtenir les données des images
    label_data = label_img.get_fdata()
    R2_data = R2_img.get_fdata()

    # Définir les labels d'intérêt
    labels_of_interest = [36, 37, 30]

    # Parcourir les labels et calculer les volumes et les valeurs moyennes
    for label in labels_of_interest:
        mask = label_data == label
        volume = np.sum(mask) * np.prod(label_img.header.get_zooms())
        mean_value = np.mean(R2_data[mask])
            #print(f'Label {label}: Volume = {volume:.2f} mm³, Valeur moyenne R2 = {mean_value:.2f}')
        # Ajouter les résultats dans une liste pour chaque directory

    for label in labels_of_interest:
        mask = label_data == label
        volume = np.sum(mask) * np.prod(label_img.header.get_zooms())
        mean_value = np.mean(R2_data[mask])
        results.append((directory, label, volume, mean_value))

# Créer un DataFrame à partir des résultats
df_results = pd.DataFrame(results, columns=['Directory', 'Label', 'Volume', 'Mean_R2'])

# Sauvegarder les résultats dans un fichier Excel
output_file = os.path.join(STUDY_PATH, 'results.xlsx')
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    df_results.to_excel(writer, index=False, sheet_name='Results')
    print(f'Les résultats ont été enregistrés dans {output_file}')

Les résultats ont été enregistrés dans /NAS/dumbo/protocoles/THY1/results.xlsx


### OLD

In [43]:

# Chemins des répertoires des images DICOM réelles et imaginaires
reel_dir = os.path.join(STUDY_PATH, 'processed_data', '20241018_BW1294', 'QSM', 'reel')
imaginaire_dir = os.path.join(STUDY_PATH, 'processed_data', '20241018_BW1294', 'QSM', 'imaginaire')

# Fonction pour lire les fichiers DICOM et extraire les données d'image
def read_dicom_images(dicom_dir):
    dicom_files = [os.path.join(dicom_dir, f) for f in os.listdir(dicom_dir) if f.endswith('.dcm')]
    dicom_files.sort()
    images = [pydicom.dcmread(f).pixel_array for f in dicom_files]
    return np.stack(images, axis=-1)

# Lire les images réelles et imaginaires
reel_images = read_dicom_images(reel_dir)
imaginaire_images = read_dicom_images(imaginaire_dir)

# Calculer les images de phase
phase_images = np.arctan2(imaginaire_images, reel_images)

# Créer un objet NIfTI
nifti_image = nib.Nifti1Image(phase_images, np.eye(4))

# Enregistrer l'image NIfTI
nifti_output_path = os.path.join(STUDY_PATH, 'processed_data', '20241018_BW1294', 'QSM', 'phase_images.nii')
nib.save(nifti_image, nifti_output_path)

print(f'Les images de phase ont été enregistrées dans {nifti_output_path}')

Les images de phase ont été enregistrées dans /NAS/dumbo/protocoles/THY1/processed_data/20241018_BW1294/QSM/phase_images.nii


In [25]:
import pydicom

dicom_path = os.path.join(DATA_PATH, '20241127_BW1302', 'T0', 'IRM', 'dicom', '3DMGET2stardef13_100u')
dicom_files = glob.glob(os.path.join(dicom_path, '*.dcm'))

for dicom_file in dicom_files:
    ds = pydicom.dcmread(dicom_file)
    image_type = ds.get('ImageType', 'Unknown')
    print(f'{dicom_file}: ImageType = {image_type}')

/NAS/dumbo/protocoles/THY1/u1172_thy1_devos_cedrick/20241127_BW1302/T0/IRM/dicom/3DMGET2stardef13_100u/MRIm723.dcm: ImageType = ['ORIGINAL', 'PRIMARY', 'OTHER']
/NAS/dumbo/protocoles/THY1/u1172_thy1_devos_cedrick/20241127_BW1302/T0/IRM/dicom/3DMGET2stardef13_100u/MRIm130.dcm: ImageType = ['ORIGINAL', 'PRIMARY', 'OTHER']
/NAS/dumbo/protocoles/THY1/u1172_thy1_devos_cedrick/20241127_BW1302/T0/IRM/dicom/3DMGET2stardef13_100u/MRIm187.dcm: ImageType = ['ORIGINAL', 'PRIMARY', 'OTHER']
/NAS/dumbo/protocoles/THY1/u1172_thy1_devos_cedrick/20241127_BW1302/T0/IRM/dicom/3DMGET2stardef13_100u/MRIm332.dcm: ImageType = ['ORIGINAL', 'PRIMARY', 'OTHER']
/NAS/dumbo/protocoles/THY1/u1172_thy1_devos_cedrick/20241127_BW1302/T0/IRM/dicom/3DMGET2stardef13_100u/MRIm128.dcm: ImageType = ['ORIGINAL', 'PRIMARY', 'OTHER']
/NAS/dumbo/protocoles/THY1/u1172_thy1_devos_cedrick/20241127_BW1302/T0/IRM/dicom/3DMGET2stardef13_100u/MRIm266.dcm: ImageType = ['ORIGINAL', 'PRIMARY', 'OTHER']
/NAS/dumbo/protocoles/THY1/u1172_t